In [1]:
%run ../yp_utils.py

# Initial setup

In [2]:
paper_pmid = 16839886
paper_name = 'slessareva_dohlman_2006' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [5]:
original_data = pd.read_excel('raw_data/hits.xlsx', sheet_name='Sheet1')

In [6]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 7 x 7


In [7]:
original_data.head()

,ystematic Name,Standard Name,Function,Gpa1Q323LResponsea,Plasmid Rescue,α Factor Max Responseb,"α Factor LogEC50, Mb ; c"
0,YOR080W,DIA2d,component of the SCF ubiquitin ligase complex;...,12.7% ± 1.2%,yes,77.8% ± 2.0%,−5.88 ± 0.06
1,YLR407W,NaN,protein of unknown function; localized to cell...,7.5% ± 0.4%,yes,97.7% ± 0.8%,−6.20 ± 0.02
2,YLR262C,YPT6,small GTPase; regulates intracellular protein ...,16.1% ± 1.4%,yes,68.2% ± 1.2%,−6.06 ± 0.04
3,YNL162W,RPL42A,cytoplasmic large ribosomal subunit,10.0% ± 0.5%,yes,78.1% ± 0.6%,−6.27 ± 0.02
4,YDR028C,REG1,cytoplasmic protein phosphatase regulator; con...,3.4% ± 0.1%,yes,85.5% ± 1.8%,−6.10 ± 0.04


In [8]:
original_data['orf'] = original_data['ystematic Name'].astype(str)

In [9]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [10]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [11]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [ystematic Name, Standard Name, Function, Gpa1Q323LResponsea, Plasmid Rescue, α Factor Max Responseb, α Factor LogEC50, Mb ;  c, orf]
Index: []


In [13]:
original_data['data'] = original_data['Gpa1Q323LResponsea'].apply(lambda x: float(x.split('%')[0])/100)

In [14]:
original_data.set_index('orf', inplace=True)

In [15]:
original_data = original_data[['data']].copy()

In [16]:
original_data = original_data.groupby(original_data.index).mean()

In [17]:
original_data.shape

(7, 1)

In [18]:
original_data.head()

,data
orf,
YBR097W,0.090
YDR028C,0.034
YLR240W,0.092
YLR262C,0.161
YLR407W,0.075


# Prepare the final dataset

In [19]:
data = original_data.copy()

In [20]:
dataset_ids = [5236]
datasets = datasets.reindex(index=dataset_ids)

In [21]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [22]:
data.head()

dataset_id,5236
data_type,value
orf,
YBR097W,0.090
YDR028C,0.034
YLR240W,0.092
YLR262C,0.161
YLR407W,0.075


## Subset to the genes currently in SGD

In [23]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 0


In [24]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,5236
,data_type,value
gene_id,orf,
292,YBR097W,0.090
2155,YDR028C,0.034
3929,YLR240W,0.092
3951,YLR262C,0.161
4097,YLR407W,0.075


# Normalize

In [25]:
data_norm = normalize_phenotypic_scores(data, has_tested=False)

In [26]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [27]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

dataset_id        5236           
data_type        value     valuez
gene_id orf                      
292     YBR097W  0.090  22.119115
2155    YDR028C  0.034   8.356110
3929    YLR240W  0.092  22.610651
3951    YLR262C  0.161  39.568639
4097    YLR407W  0.075  18.432596

# Print out

In [28]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [29]:
from IO.save_data_to_db3 import *

In [30]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/1 [00:00<?, ?it/s]

Deleting all datasets for PMID 16839886...
Inserting the new data...


100%|██████████| 1/1 [00:00<00:00, 13.34it/s]

Updating the data_modified_on field...
